In [ ]:
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

# Пути к тестовым данным
test_img_dir = 'E:/Yandex_prakt/1_Proj_segment/stud_dataset/images/test'
test_ann_dir = 'E:/Yandex_prakt/1_Proj_segment/stud_dataset/annotations/test'

# Используем os.path.join для корректных путей
test_img_dir = os.path.normpath(test_img_dir)
test_ann_dir = os.path.normpath(test_ann_dir)

print(f"Путь к изображениям: {test_img_dir}")
print(f"Путь к маскам: {test_ann_dir}")

# Проверим, существуют ли директории
if not os.path.exists(test_ann_dir):
    print(f"❌ Директория не существует: {test_ann_dir}")
    # Попробуем альтернативные пути
    possible_paths = [
        'E:/Yandex_prakt/1_Proj_segment/stud_dataset/annotations/test',
        'E:/Yandex_prakt/1_Proj_segment/stud_dataset/annotations/val',
        'E:/Yandex_prakt/1_Proj_segment/stud_dataset/masks/test',
        'E:/Yandex_prakt/1_Proj_segment/stud_dataset/gt/test'
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            test_ann_dir = os.path.normpath(path)
            print(f"✅ Найдена директория: {test_ann_dir}")
            break
else:
    print(f"✅ Директория существует: {test_ann_dir}")

# Найти все файлы масок (поддерживаем разные расширения)
mask_extensions = ['*.png', '*.jpg', '*.jpeg', '*.bmp', '*.tiff']
mask_files = []

for ext in mask_extensions:
    pattern = os.path.join(test_ann_dir, ext)
    files = glob.glob(pattern)
    mask_files.extend(files)
    if files:
        print(f"Найдено {len(files)} файлов с расширением {ext}")

print(f"\nВсего найдено {len(mask_files)} файлов масок")

if len(mask_files) == 0:
    print("\n❌ Файлы масок не найдены! Покажу содержимое директории:")
    
    # Покажем что есть в директории
    if os.path.exists(test_ann_dir):
        for item in os.listdir(test_ann_dir):
            item_path = os.path.join(test_ann_dir, item)
            if os.path.isdir(item_path):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")
    
    # Посмотрим в родительской директории
    parent_dir = os.path.dirname(test_ann_dir)
    print(f"\nСодержимое родительской директории {parent_dir}:")
    if os.path.exists(parent_dir):
        for item in os.listdir(parent_dir):
            item_path = os.path.join(parent_dir, item)
            if os.path.isdir(item_path):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")
else:
    print("\n=== АНАЛИЗ ТЕСТОВЫХ ДАННЫХ ===")
    
    # Берем только первые 50 для скорости
    mask_files_to_check = mask_files[:50]
    print(f"Проверяем первые {len(mask_files_to_check)} масок...")
    
    dog_images = []
    cat_images = []
    mixed_images = []
    empty_images = []
    
    for idx, mask_file in enumerate(mask_files_to_check):
        try:
            # Загружаем маску
            mask = np.array(Image.open(mask_file))
            
            # Статистика по пикселям
            dog_pixels = np.sum(mask == 2)
            cat_pixels = np.sum(mask == 1)
            bg_pixels = np.sum(mask == 0)
            
            filename = os.path.basename(mask_file)
            
            # Проверяем значения
            unique_values = np.unique(mask)
            valid_values = [v for v in unique_values if v in [0, 1, 2]]
            
            if len(valid_values) != len(unique_values):
                print(f"⚠️  {filename}: неожиданные значения {set(unique_values) - set([0,1,2])}")
            
            # Классифицируем
            if dog_pixels > 0 and cat_pixels > 0:
                mixed_images.append(filename)
                status = "ОБА КЛАССА"
            elif dog_pixels > 0:
                dog_images.append(filename)
                status = "ТОЛЬКО СОБАКА"
            elif cat_pixels > 0:
                cat_images.append(filename)
                status = "ТОЛЬКО КОШКА"
            else:
                empty_images.append(filename)
                status = "ТОЛЬКО ФОН"
            
            # Показываем только если есть животные
            if dog_pixels > 0 or cat_pixels > 0:
                print(f"{idx:3d}. {filename:25s} | Dog: {dog_pixels:5d} | Cat: {cat_pixels:5d} | {status}")
                
        except Exception as e:
            print(f"Ошибка при обработке {mask_file}: {e}")
    
    print("\n" + "="*80)
    print("=== ПРЕДВАРИТЕЛЬНЫЕ РЕЗУЛЬТАТЫ (первые 50 изображений) ===")
    print(f"Всего проверено: {len(mask_files_to_check)}")
    print(f"\nРаспределение:")
    print(f"1. Только собаки: {len(dog_images)}")
    print(f"2. Только кошки: {len(cat_images)}")
    print(f"3. Оба класса: {len(mixed_images)}")
    print(f"4. Только фон: {len(empty_images)}")
    
    # Покажем несколько примеров
    examples = []
    
    if dog_images:
        examples.append(("Только собака", dog_images[0]))
    if cat_images:
        examples.append(("Только кошка", cat_images[0]))
    if mixed_images:
        examples.append(("Оба класса", mixed_images[0]))
    if empty_images:
        examples.append(("Только фон", empty_images[0]))
    
    if examples:
        print("\n=== ВИЗУАЛИЗАЦИЯ ПРИМЕРОВ ===")
        
        fig, axes = plt.subplots(len(examples), 2, figsize=(10, 4*len(examples)))
        if len(examples) == 1:
            axes = axes.reshape(1, -1)
        
        for i, (title, filename) in enumerate(examples):
            # Загружаем маску
            mask_path = os.path.join(test_ann_dir, filename)
            mask = np.array(Image.open(mask_path))
            
            # Загружаем изображение
            img_filename = filename.replace('.png', '.jpg').replace('.PNG', '.jpg')
            img_path = os.path.join(test_img_dir, img_filename)
            
            if os.path.exists(img_path):
                img = np.array(Image.open(img_path))
                axes[i, 0].imshow(img)
                axes[i, 0].set_title(f"Изображение: {title}")
            else:
                # Попробуем другие расширения
                found = False
                for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                    img_path = os.path.join(test_img_dir, filename.replace('.png', ext))
                    if os.path.exists(img_path):
                        img = np.array(Image.open(img_path))
                        axes[i, 0].imshow(img)
                        axes[i, 0].set_title(f"Изображение: {title}")
                        found = True
                        break
                
                if not found:
                    axes[i, 0].text(0.5, 0.5, f"Изображение не найдено\n{filename}", 
                                  ha='center', va='center', transform=axes[i, 0].transAxes)
            
            axes[i, 0].axis('off')
            
            # Показываем маску
            axes[i, 1].imshow(mask, cmap='jet', vmin=0, vmax=2)
            axes[i, 1].set_title(f"Маска: {title}")
            axes[i, 1].axis('off')
            
            # Подпишем цвета
            colors = {0: 'черный', 1: 'синий', 2: 'красный'}
            mask_summary = []
            for val in [0, 1, 2]:
                count = np.sum(mask == val)
                if count > 0:
                    mask_summary.append(f"{colors[val]}({val}): {count}")
            
            axes[i, 1].text(0.02, 0.98, "\n".join(mask_summary),
                           transform=axes[i, 1].transAxes,
                           verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
    
    # Критическая проверка для проблемы с Dog
    print("\n" + "="*80)
    print("=== КРИТИЧЕСКИЙ АНАЛИЗ ДЛЯ КЛАССА DOG ===")
    
    if len(dog_images) == 0 and len(mixed_images) == 0:
        print("❌❌❌ СЕРЬЁЗНАЯ ПРОБЛЕМА ❌❌❌")
        print("В проверенных тестовых данных НЕТ НИ ОДНОЙ СОБАКИ!")
        print("Класс Dog в ground truth полностью отсутствует.")
        print("\nПричины Dice=0.0000 для Dog:")
        print("1. В GT нет пикселей с классом 2 (Dog)")
        print("2. Модель пытается предсказать собак (6,829 пикселей)")
        print("3. Все предсказания - ложноположительные (FP)")
        print("4. TP = 0, FN = 0, FP > 0 → Dice = IoU = 0")
    else:
        print(f"✅ Собаки присутствуют в тестовых данных")
        print(f"   Изображений только с собаками: {len(dog_images)}")
        print(f"   Изображений с обоими классами: {len(mixed_images)}")